Create dataset from swiss card images

In [1]:
import math
import numpy as np
import cv2 as cv
from pathlib import Path
import random
from itertools import cycle

from jassair.utils import get_dataset_path, Datasets, get_class_for_name
from jassair.synthetic_data import rotate_image, add_drop_shadow, augment_image, Vignette, LightSpots, RotateMult90, LightingGradient, ColorJitter, NoiseNormal, GaussianBlur, PerspectiveWarp, WhiteBalanceShift

## Create background images

In [2]:
#dest_path = get_dataset_path(Datasets.BACKGROUNDS)

In [3]:
target_x = 640

source_path = Path("data/raw_backgrounds")

for i, image_path in enumerate(source_path.glob("*"), 1):
    image = cv.imread(image_path)
    
    h, w = image.shape[:2]
    h_2, w_2 = h // 2, w // 2
    x = min(h, w)
    x_2 = x // 2
    
    if x < target_x:
        raise ValueError(f"Image {image_path} is too small!, {image.shape[:2]}")
    
    h_start = h_2 - x_2
    h_stop = h_2 + x_2
    w_start = w_2 - x_2
    w_stop = w_2 + x_2        
    
    image = image[h_start:h_stop, w_start:w_stop]

    image = cv.resize(image, (target_x, target_x))
    cv.imwrite(f"{dest_path}/background_{i}.png", image) 

NameError: name 'dest_path' is not defined

## Read images

In [4]:
dataset_path = get_dataset_path(Datasets.CARD_TEMPLATE)

In [5]:
FOREGROUND_IMAGES: list[tuple[np.ndarray, int]] = []

In [6]:
for image_path in dataset_path.glob("*"):
    image = cv.imread(image_path, cv.IMREAD_UNCHANGED)
    label = int(get_class_for_name(image_path.stem))
    
    FOREGROUND_IMAGES.append((image, label))
print(len(FOREGROUND_IMAGES))

36


In [7]:
BACKGROUND_IMAGES : list[np.ndarray] = []

In [8]:
for image_path in get_dataset_path(Datasets.BACKGROUNDS).glob("*"):
    BACKGROUND_IMAGES.append(cv.imread(image_path))
print(len(BACKGROUND_IMAGES))

99


## YOLO Dataset description

In [9]:
DESCRIPTION = """train: ./train/images
val: ./valid/images
test: ./test/images

nc: 36
names: ['Eichel 10', 'Eichel 6', 'Eichel 7', 'Eichel 8', 'Eichel 9', 'Eichel Ass', 'Eichel Konig', 'Eichel Ober', 'Eichel Under', 'Rose 10', 'Rose 6', 'Rose 7', 'Rose 8', 'Rose 9', 'Rose Ass', 'Rose Konig', 'Rose Ober', 'Rose Under', 'Schelle 10', 'Schelle 6', 'Schelle 7', 'Schelle 8', 'Schelle 9', 'Schelle Ass', 'Schelle Konig', 'Schelle Ober', 'Schelle Under', 'Schilte 10', 'Schilte 6', 'Schilte 7', 'Schilte 8', 'Schilte 9', 'Schilte Ass', 'Schilte Konig', 'Schilte Ober', 'Schilte Under']
"""

## Create synthetic images

In [10]:
BACKGROUND_AUGMENTS = [
    RotateMult90(),
    ColorJitter(hue=0.1)
]

In [35]:
FINAL_AUGMENT = [
    ColorJitter(0.3, 0.3, 0.2, 0.1),
    LightingGradient(0.7),
    LightSpots(),
    Vignette(0.3)
]

In [36]:
class PlacementError(Exception): ...

In [37]:
def iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1b, y1b, x2b, y2b = box2
    xi1, yi1 = max(x1, x1b), max(y1, y1b)
    xi2, yi2 = min(x2, x2b), min(y2, y2b)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2b - x1b) * (y2b - y1b)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area else 0

In [38]:
def generate_synthetic_card_scene(bg: np.ndarray, card_images: list[np.ndarray], angles: list[float], scale: float = 0.7, iou_threshold: float = 0.1) -> tuple[np.ndarray, list[str]]:
    """
    Generates a realistic card scene on a background.
    Returns final image and YOLO-format annotations.
    """
    bg = augment_image(bg, BACKGROUND_AUGMENTS)
    h_bg, w_bg = bg.shape[:2]
    
    shadowed = [add_drop_shadow(img, (50, 50), 20, 0.8) for img in card_images]
    rotated = [rotate_image(img, angle) for img, angle in zip(shadowed, angles)]
    max_card = max(rotated, key=lambda x: sum(x.shape[:2]))
    h_max, w_max = max_card.shape[:2]
    ratio = min(h_bg / h_max, w_bg / w_max)
    
    processed_cards = []
    for img in rotated:
        h, w = img.shape[:2]
        processed_cards.append((cv.resize(img, (int(w * ratio * scale), int(h * ratio * scale))), (h, w)))

    boxes = []
    placed = []
    for card, (bh_card, bw_card) in processed_cards:
        h_card, w_card = card.shape[:2]
        max_attempts = 30
        for _ in range(max_attempts):
            x = np.random.randint(0, w_bg - w_card)
            y = np.random.randint(0, h_bg - h_card)

            # Check for overlap
            bbox = (x, y, x + w_card, y + h_card)
            if all(iou(bbox, existing) < iou_threshold for existing in placed):
                placed.append(bbox)
                break
        else:
            raise PlacementError("Unable to place card without overlap")

        roi = bg[y:y+h_card, x:x+w_card]
        alpha = card[:, :, 3] / 255
        alpha_3ch = np.dstack([alpha]*3)
        card_rgb = card[:, :, :3]
        blended = (roi * (1 - alpha_3ch) + card_rgb * alpha_3ch)
        bg[y:y+h_card, x:x+w_card] = blended

        # YOLO box
        cx = (x + w_card / 2) / w_bg
        cy = (y + h_card / 2) / h_bg
        bw = w_card / w_bg
        bh = h_card / h_bg
        boxes.append(f"{cx} {cy} {bw} {bh}")

    final = augment_image(bg, FINAL_AUGMENT)
    return final, boxes

In [39]:
def create_synthetic_images(num_images: int, image_dest: Path, label_dest: Path, set_name: str, cards_per_image: tuple[int, int], overlap_threshold: float):
    images = iter(())
    for i in range(num_images):
        if not i % 128:
            random.shuffle(FOREGROUND_IMAGES)
            images = cycle(FOREGROUND_IMAGES)
        
        bg = random.choice(BACKGROUND_IMAGES).copy()
        
        num_cards = random.randint(*cards_per_image)
        cards = [next(images) for _ in range(num_cards)]
        angles = [random.gauss(0, 45.0) for _ in range(num_cards)]
        labels = [card[1] for card in cards]
        cards = [card[0].copy() for card in cards]
        scale = random.uniform(0.5 / max(1.0, math.log2(num_cards * (2 - overlap_threshold))), 0.8 / max(1.0, math.log2(num_cards * (2 - overlap_threshold))))
    
        # Overlay object onto background
        synthetic_image, boxes = generate_synthetic_card_scene(bg, cards, angles, scale, overlap_threshold)
    
        # Save image
        cv.imwrite(f"{image_dest}/{set_name}_{i}.png", synthetic_image)
        with (label_dest / f"{set_name}_{i}.txt").open("w+", encoding="utf-8") as f:
            for label, box in zip(labels, boxes):
                f.write(f"{label} {box}\n")

In [40]:
def create_synthetic_dataset(dataset_dest: Path, num_train: int, num_val: int, num_test: int, cards_per_image: tuple[int, int], max_overlap: float):
    # Create training data
    image_target = dataset_dest / "train" / "images"
    image_target.mkdir(parents=True, exist_ok=True)
    label_target = dataset_dest / "train" / "labels"
    label_target.mkdir(parents=True, exist_ok=True)
    create_synthetic_images(num_train, image_target, label_target, "train", cards_per_image, max_overlap)
    
    # Create validation data
    image_target = dataset_dest / "valid" / "images"
    image_target.mkdir(parents=True, exist_ok=True)
    label_target = dataset_dest / "valid" / "labels"
    label_target.mkdir(parents=True, exist_ok=True)
    create_synthetic_images(num_val, image_target, label_target, "valid", cards_per_image, max_overlap)
    
    # Create test data
    image_target = dataset_dest / "test" / "images"
    image_target.mkdir(parents=True, exist_ok=True)
    label_target = dataset_dest / "test" / "labels"
    label_target.mkdir(parents=True, exist_ok=True)
    create_synthetic_images(num_test, image_target, label_target, "test", cards_per_image, max_overlap)
    
    # Write data.yaml
    with (dataset_dest / "data.yaml").open("w+", encoding="utf-8") as f:
        f.write(DESCRIPTION)
        

In [41]:
num_train = 10
num_valid = 0
num_test = 0
overlap = 0.1
cards_per_image = (1, 1)

In [42]:
random.seed(42)
create_synthetic_dataset(get_dataset_path(Datasets.SYNTHETIC_SINGLE), num_train, num_valid, num_test, cards_per_image, overlap)

print("Synthetic dataset created successfully!")

Synthetic dataset created successfully!
